In [36]:
# !ollama pull llama3.2:1b-instruct-fp16


In [1]:
import requests
from bs4 import BeautifulSoup
import os
import json
import re
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_core.tools import tool


# The @tool decorator from langchain_core.tools to define custom tools in LangChain.

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'. Both 'x' and 'y' are numbers."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'. Both 'x' and 'y' are numbers."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'. Both 'x' and 'y' are numbers."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'. Both 'x' and 'y' are numbers."""
    return y - x

With the @tool decorator our function is turned into a StructuredTool object, which we can see below:

In [4]:
add

StructuredTool(name='add', description="Add 'x' and 'y'. Both 'x' and 'y' are numbers.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x1077c7c40>)

In [5]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'. Both 'x' and 'y' are numbers."


In [6]:
# The args_schema attribute in LangChain tools allows you to define a more structured schema for the arguments that the tool accepts. 

add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'. Both 'x' and 'y' are numbers.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'. Both 'x' and 'y' are numbers.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [8]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [9]:
exponentiate.func(**llm_output_dict)

25

### Create an Agent

LangChain Epression Language (LCEL) to construct the agent.

In [10]:
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)

SyntaxError: invalid syntax (3950850003.py, line 2)

**MessagesPlaceholder** is a placeholder in a prompt template that reserves space for a list of Message objects. These Message objects typically represent the conversation history or other dynamic content that will be inserted into the prompt at runtime.

In [11]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages([
#     ("system", (
#         "You're a helpful assistant. Using the tools provided you must answer the "
#         "user's questions. To use the tool you must always provide the correct JSON "
#         "format for the tool input. For example, if adding two numbers you would use "
#         "the `add` tool, passing the two numbers to the `x` and `y` parameters."
#     )),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("human", "{input}"),
#     ("placeholder", "{agent_scratchpad}"),
# ])

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("ai", "Scratchpad: {agent_scratchpad}"),
])

In [12]:

# !pip install langchain
# !pip install langchain-core

# !pip install langchain-community

# !pip install langchain_ollama



In [13]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older ConversationBufferMemory class rather than the newer RunnableWithMessageHistory — the reason being that we will also be using the older create_tool_calling_agent and AgentExecutor method and class.


In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

/var/folders/hb/_y2tps0n0m77ytw9p5xk3kx41dbrb_/T/ipykernel_66088/287820297.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Initialize the agent

- llm: as already defined
- tools: to be defined (just a list of our previously defined tools)
- prompt: as already defined
- memory: as already defined

In [15]:
from langchain.agents import create_tool_calling_agent

tools = [add, multiply, exponentiate, subtract]

agent = create_tool_calling_agent(
    llm,
    tools,
    prompt,
)

In [16]:
# !pip install colab-xterm

# !ollama serve &

# !ollama pull llama3.2:1b-instruct-fp16

# !curl http://127.0.0.1:11434

# !ollama list

In [17]:
agent.invoke({
    "input": "what is 10 multiplied by 7?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

AgentFinish(return_values={'output': ''}, log='')

In [19]:
# We use the AgentExecutor class to handle the execution loop:

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [20]:
agent_executor.invoke({
    "input": "what is 10 multiplied by 7?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'what is 10 multiplied by 7?',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={})],
 'output': ''}

In [21]:
agent_executor.invoke({
    "input": "My name is Josh",
    "chat_history": memory
})



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'My name is Josh',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={})],
 'output': ''}

In [22]:
9+10-(4*2)**3


-493

In [23]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={})],
 'output': ''}

### SerpAPI Weather Agent

In [63]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

In [64]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

In [65]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"
    
@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [67]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [68]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [71]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time in London right now? "
        "How is the weather where I am? Please give me the degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `London`


The current date and time in London is Wednesday, March 8th, 2023, 20:43:31 UTC (Coordinated Universal Time). 

As for the weather, I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the current weather in London.

You can check the weather forecast for London on websites like AccuWeather, Weather.com, or the Met Office (the UK's national weather service). These websites provide up-to-date weather forecasts and conditions for various locations around the world, including London.

Alternatively, you can also download a weather app on your smartphone to get the current weather conditions in London. Some popular weather apps include Dark Sky, Weather Underground, and The Weather Channel.

> Finished chain.


In [72]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

The current date and time in London is Wednesday, March 8th, 2023, 20:43:31 UTC (Coordinated Universal Time). 

As for the weather, I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the current weather in London.

You can check the weather forecast for London on websites like AccuWeather, Weather.com, or the Met Office (the UK's national weather service). These websites provide up-to-date weather forecasts and conditions for various locations around the world, including London.

Alternatively, you can also download a weather app on your smartphone to get the current weather conditions in London. Some popular weather apps include Dark Sky, Weather Underground, and The Weather Channel.

## LangChain Agent Executor

Reason + Action (ReAct) agents use iterative reasoning and action steps to incorporate chain-of-thought and tool-use into their execution. During the reasoning step the LLM generates what steps to take to answer the query. Next, the LLM generates the action input, which our code logic parses into a tool call.

To add tools to our LLM, we will use the bind_tools method within the LCEL constructor, which will take our tools and add them to the LLM. We'll also include the tool_choice="any" argument to bind_tools, which tells the LLM that it MUST use a tool, ie it cannot provide a final answer directly

In [24]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [25]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x123263240>)

In [26]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [27]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [28]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [29]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [30]:
exponentiate.func(**llm_output_dict)

25

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("ai", "Scratchpad: {agent_scratchpad}"),
])

In [32]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)
bound_llm = llm.bind_tools(tools, tool_choice="any")

# llm
# bound_llm


In [35]:
from langchain.tools import Tool

# Define a tool
def search_tool(query: str) -> str:
    return f"Search results for: {query}"

tools = [
    Tool(
        name="Search",
        func=search_tool,
        description="Useful for searching the web or answering questions about current events."
    )
]

In [36]:
from langchain_core.runnables.base import RunnableSerializable

# tools = [add, subtract, multiply, exponentiate]

# print(tools)

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
    }
    | prompt
    | bound_llm
)

# agent

{
  input: RunnableLambda(...),
  chat_history: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMe

In [39]:
input_data = {
    "input": "What is 10+10?",
    "chat_history": [],
    "agent_scratchpad": ""
}

out = agent.invoke(input_data)
out

AIMessage(content='20', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-03-08T21:11:34.224951Z', 'done': True, 'done_reason': 'stop', 'total_duration': 152542584, 'load_duration': 29312250, 'prompt_eval_count': 132, 'prompt_eval_duration': 112000000, 'eval_count': 2, 'eval_duration': 10000000, 'message': Message(role='assistant', content='20', images=None, tool_calls=None)}, id='run-57acfcc2-e6cd-4602-951c-e48923320c44-0', usage_metadata={'input_tokens': 132, 'output_tokens': 2, 'total_tokens': 134})

In [40]:
out.tool_calls

[]

In [122]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

In [ ]:
tool_output = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
tool_output